# SETTINGS

In [1]:
############ LIBRARIES

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
%matplotlib inline

import os
import time
import random
import multiprocessing
import pickle

import scipy.stats

import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.decomposition import PCA, FastICA, FactorAnalysis
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
############ HELPER FUNCTIONS

import functions
import importlib
importlib.reload(functions)
from functions import *

In [3]:
############ RANDOMNESS

# seed function
def seed_everything(seed = 23):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 23
seed_everything(seed)

# DATA PREPARATION

In [4]:
# import CSV
df = pd.read_pickle('../input/data_v9.pkl')
print(df.shape)

(1097231, 511)


In [ ]:
# target variable
target = 'isFraud'

In [ ]:
# partitioning
full_train = df[df[target].isnull() == False].reset_index(drop = True)
full_test  = df[df[target].isnull() == True].reset_index(drop = True)
print(full_train.shape)
print(full_test.shape)
del df

(590540, 511)
(506691, 511)


# MODELING

### PARAMETERS

In [ ]:
############ FEAUTERS

# drop bad features
excluded_feats = ['isFraud',
                  'TransactionID', 'TransactionDT',
                  'bank_type',
                  'uid1', 'uid2', 'uid3', 'uid4', 'uid5', 
                  'DT', 'DT_D', 'DT_W', 'DT_M', 'DT_hour', 
                  'DT_day_week', 'DT_day_month', 
                  'DT_D_total', 'DT_W_total', 'DT_M_total']
full_features = [f for f in full_train.columns if f not in excluded_feats]
print(full_train[full_features].shape)

(590540, 504)


In [ ]:
############ PARAMETERS

# cores
cores = 24

# cross-validation
num_folds = 6
shuffle   = True

# number of trees
max_rounds = 10000
stopping   = 200
verbose    = 250

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metric':            'auc',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.9,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0,
    'min_child_weight':  0.1,
    'min_child_samples': 20,
    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.01,
    'max_depth':         5,
    'num_leaves':        100,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          cores,
    'random_state':      seed,
}

lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':cores,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':max_rounds,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': seed,
                } 

# data partitinoing
#folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)
folds = GroupKFold(n_splits = num_folds)

# SMOTE settings
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state = seed, n_jobs = cores, sampling_strategy = 0.1)

In [ ]:
############ PLACEHOLDERS

# placeholders
clfs = []
importances = pd.DataFrame()

# predictions
preds_test = np.zeros(full_test.shape[0])
preds_oof  = np.zeros(full_train.shape[0])

### CROSS-VALIDATION

In [ ]:
cv_start  = time.time()
for pcd in full_train.ProductCD.unique():
    
    print('-'*15)
    print('-'*15)
    print('PRODUCT CD = ' + str(pcd))
    print('-'*15)
    print('-'*15)
    
    train = full_train[full_train.ProductCD == pcd]
    test  = full_test[full_test.ProductCD   == pcd]

    idx1 = train.index
    idx2 = test.index
        
    train.reset_index(drop = True, inplace = True)
    test.reset_index(drop  = True, inplace = True)
    
    y = train[target]
    del train[target], test[target]
    
    features = full_features.copy()
    for var in train.columns:
        if train[var].isnull().sum() == len(train):
            del train[var], test[var]
            if var in features:
                features.remove(var)


    ############ CROSS-VALIDATION LOOP
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y, groups = train['DT_M'])):

        # data partitioning
        trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
        val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
        test_x       = test[features]

        # label encoding
        trn_x, val_x, test_x = label_encoding(trn_x, val_x, test_x)

        # print data dimensions
        print('Data shape:', trn_x.shape, val_x.shape)

        # train lightGBM
        clf = lgb.LGBMClassifier(**lgb_params) 
        clf = clf.fit(trn_x, trn_y, 
                      eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                      eval_metric           = 'auc', 
                      early_stopping_rounds = stopping,
                      verbose               = verbose)
        clfs.append(clf)

        # find the best iteration
        best_iter = clf.best_iteration_

        # save predictions
        preds_oof[idx1[val_idx]] = clf.predict_proba(val_x,  num_iteration = best_iter)[:, 1]
        preds_test[idx2]        += clf.predict_proba(test_x, num_iteration = best_iter)[:, 1] / folds.n_splits 

        # importance
        fold_importance_df               = pd.DataFrame()
        fold_importance_df['Feature']    = features
        fold_importance_df['Importance'] = clf.feature_importances_
        fold_importance_df['Fold']       = n_fold + 1
        importances                      = pd.concat([importances, fold_importance_df], axis = 0)

        # print performance
        print('--------------------------------')
        print('FOLD%2d: AUC = %.6f' % (n_fold + 1, roc_auc_score(y[val_idx], preds_oof[idx1[val_idx]])))
        print('--------------------------------')
        print('')

        # clear memory
        del trn_x, trn_y, val_x, val_y
        gc.collect()


    # print overall performance    
    cv_perf = roc_auc_score(y, preds_oof[idx1])
    print('--------------------------------')
    print('- OOF AUC = %.6f' % cv_perf)
    print('--------------------------------')
    
    
print('CV TIME = {:.2f} min'.format((time.time() - cv_start) / 60))

---------------
---------------
PRODUCT CD = W
---------------
---------------
Data shape: (358122, 307) (81548, 307)
Training until validation scores don't improve for 200 rounds.
[250]	training's auc: 0.991302	valid_1's auc: 0.897745
[500]	training's auc: 0.99985	valid_1's auc: 0.912228
[750]	training's auc: 0.999995	valid_1's auc: 0.915903
[1000]	training's auc: 1	valid_1's auc: 0.917197
[1250]	training's auc: 1	valid_1's auc: 0.918132
Early stopping, best iteration is:
[1190]	training's auc: 1	valid_1's auc: 0.917848
--------------------------------
FOLD 1: AUC = 0.917848
--------------------------------

Data shape: (363531, 307) (76139, 307)
Training until validation scores don't improve for 200 rounds.
[250]	training's auc: 0.989435	valid_1's auc: 0.87855
[500]	training's auc: 0.999769	valid_1's auc: 0.89183
[750]	training's auc: 0.999994	valid_1's auc: 0.894822
[1000]	training's auc: 1	valid_1's auc: 0.896599
[1250]	training's auc: 1	valid_1's auc: 0.897168
Early stopping, best

### EVALUATION

In [ ]:
############ RECHECK PERFORMANCE  

# check performance
y = full_train[target]
print(np.round(roc_auc_score(y, preds_oof), 5))
cv_perf = roc_auc_score(y, preds_oof)


############ TRACK RESULTS


In [ ]:
########## CONFUSION MATRIX

# construct confusion matrx
cm = confusion_matrix(y, np.round(preds_oof))
cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]

# plot matrix
fig, ax = plt.subplots(figsize = (10, 4))
sns.heatmap(cm, cmap = 'Blues', annot = True, lw = 0.5)
ax.set_xlabel('Prediction')
ax.set_ylabel('Ground Truth')
ax.set_aspect('equal')

# SUBMISSION

In [ ]:
# file name
model = 'lgb_v9p'
perf  = str(round(cv_perf, 6))[2:7]
name  = model + '_' + perf
name

In [ ]:
# export OOF preds
oof = pd.DataFrame({'TransactionID': full_train['TransactionID'], 'isFraud': preds_oof})
oof.to_csv('../oof_preds/' + str(name) + '.csv', index = False)
oof.head()

In [ ]:
# export submission
sub = pd.DataFrame({'TransactionID': full_test['TransactionID'], 'isFraud': preds_test})
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.head()

In [ ]:
# check rank correlation with the best submission
from scipy.stats import spearmanr
sub  = sub.sort_values('TransactionID')
best = pd.read_csv("../submissions/BlendSolution.csv")
best = best.sort_values('TransactionID')
spearmanr(sub.isFraud, best.isFraud)